<a href="https://colab.research.google.com/github/nyp-sit/it3103/blob/main/week13/putting_it_all_together.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practical 4c - Putting It All Together

In this final part, we will take the models that we have trained and use them to recognize an intent and the entities and building simple responses to that. 

Before starting, click on the Colab's Runtime > Manage Sessions menu. Click the "TERMINATE OTHER SESSIONS" button.  

Then, run the following cells to download the models (after training them to our intent and token classification tasks) and install the necessary libraries. 

The reason we are doing this is because downloading the models that you have trained from Colab is VERY slow. So we've already saved a copy of our own trained models and uploaded it to a public server on Amazon for download.


In [1]:
!wget https://nyp-aicourse.s3.ap-southeast-1.amazonaws.com/pretrained-models/intent_model.zip
!wget https://nyp-aicourse.s3.ap-southeast-1.amazonaws.com/pretrained-models/token_model.zip
!unzip intent_model.zip
!unzip token_model.zip


--2021-06-18 15:32:36--  https://nyp-aicourse.s3.ap-southeast-1.amazonaws.com/pretrained-models/intent_model.zip
Resolving nyp-aicourse.s3.ap-southeast-1.amazonaws.com (nyp-aicourse.s3.ap-southeast-1.amazonaws.com)... 52.219.128.131
Connecting to nyp-aicourse.s3.ap-southeast-1.amazonaws.com (nyp-aicourse.s3.ap-southeast-1.amazonaws.com)|52.219.128.131|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 246825147 (235M) [application/zip]
Saving to: ‘intent_model.zip’

intent_model.zip    100%[===================>] 235.39M  17.8MB/s    in 15s     

2021-06-18 15:32:52 (15.5 MB/s) - ‘intent_model.zip’ saved [246825147/246825147]

--2021-06-18 15:32:52--  https://nyp-aicourse.s3.ap-southeast-1.amazonaws.com/pretrained-models/token_model.zip
Resolving nyp-aicourse.s3.ap-southeast-1.amazonaws.com (nyp-aicourse.s3.ap-southeast-1.amazonaws.com)... 52.219.128.175
Connecting to nyp-aicourse.s3.ap-southeast-1.amazonaws.com (nyp-aicourse.s3.ap-southeast-1.amazonaws.com)|52.21

Next, run the following to install a specific version of the HuggingFace Transformers library.

Our model was trained against this version of the library, so it is advisable to use the same version for prediction / inference.

In [5]:
!pip install transformers

     |████████████████████████████████| 2.5MB 8.2MB/s 
     |████████████████████████████████| 3.3MB 52.0MB/s 
     |████████████████████████████████| 901kB 53.1MB/s 


## Section 1 - Inferring Intent

In this section, we declare the codes to infer intent based on a single line of input text.

In [13]:
# Import the necessary libraries
#
from transformers import (
    AutoTokenizer, 
    TFAutoModelForTokenClassification,
    TFAutoModelForSequenceClassification
)

import numpy as np
import tensorflow as tf


# Create the DistilBERT tokenizer
#
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')


# Create a list of unique labels that we will recognize.
#
sentence_labels = [
              "others",
              "atis_abbreviation",
              "atis_aircraft",
              "atis_airfare",
              "atis_airline",
              "atis_flight",
              "atis_flight_time",
              "atis_greeting",
              "atis_ground_service",
              "atis_quantity",
              "atis_yes",
              "atis_no"]

# Define a function to perform inference on a single input text.
# 
def infer_intent(model, text):
    # Passes the text into the tokenizer
    #
    input = tokenizer(text, truncation=True, padding=True, return_tensors="tf")

    # Sends the result from the tokenizer into our classification model
    #
    output = model(input)
    pred_label = np.argmax(tf.nn.softmax(output.logits, axis=-1))

    # Return the result to the caller
    #
    return sentence_labels[pred_label]


# Load the saved model file
#
intent_model = TFAutoModelForSequenceClassification.from_pretrained('intent_model')



Some layers from the model checkpoint at intent_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at intent_model and are newly initialized: ['dropout_79']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Run the following cell to test the codes that infers the intent. 

In [14]:
infer_intent(intent_model, "How much is the ticket to fly to New York")

'atis_airfare'

## Section 2 - Inferring Entity

In this section, we declare the codes to infer entities for each individual word in a line of text. The entities are then constructed and returned to the caller.


In [25]:
# Define a list of unique labels that we will recognized
#
token_labels = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

# Define the function to infer the individual tokens
#

def infer_tokens(model, text):
    encodings = tokenizer([text], 
                          is_split_into_words=True, 
                          padding=True, 
                          truncation=True, 
                          return_offsets_mapping=True, 
                          return_tensors="tf")

    label_mapping = [0] * len(encodings.offset_mapping[0])
    for i, offset in enumerate(encodings.offset_mapping[0]):
        if encodings.offset_mapping[0][i][0] == 0 and encodings.offset_mapping[0][i][1] != 0:
            label_mapping[i] = 1

    encodings.pop("offset_mapping")
   
    output = token_model(encodings)[0]

    cur_index = -1
    result_tokens = []
    result_texts = []

    for i in range(output.shape[1]):
        if label_mapping[i] == 1:
            result_tokens.append(np.argmax(output[0][i]).item())
            result_texts.append(text[i])
            cur_index += 1
        else:
            if cur_index >= 0 and text[i] != "[CLS]" and text[i] != "[SEP]":
                result_texts[cur_index] += text[i].replace("##", "")


    return result_tokens, result_texts

# Define the function to combine individual tokens into a dictionary
#
def infer_combined_tokens(token_model, text):
    result = {
        "PER" : [],
        "LOC" : [],
        "ORG" : [],
        "MISC" : []
    }

    result_tokens, result_texts = infer_tokens(token_model, text)

    current_token_label = ""
    current_result_index = -1;    

    for i in range(len(result_tokens)):
        if token_labels[result_tokens[i]].startswith("B-"):
            current_token_label = token_labels[result_tokens[i]].replace("B-", "")
            result[current_token_label].append(result_texts[i])
            current_result_index = len(result[current_token_label]) - 1
        elif token_labels[result_tokens[i]].startswith("I-"):
            result[current_token_label][current_result_index] += " " + result_texts[i]
    
    return result

# Load the saved model file
#



In [26]:
token_model = TFAutoModelForTokenClassification.from_pretrained('token_model')

Some layers from the model checkpoint at token_model were not used when initializing TFDistilBertForTokenClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForTokenClassification were not initialized from the model checkpoint at token_model and are newly initialized: ['dropout_159']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Run the following cell to test the codes that extracts and combines all the entities for us.

In [27]:
infer_combined_tokens(token_model, "How much is the ticket to fly to New York")

{'LOC': [], 'MISC': [], 'ORG': [], 'PER': []}

## Section 3 - Implementing Logic for Our Chatbot

In this section, let's implement some very basic logic for our chatbot. We will make use of the two functions that we wrote above.

You can implement some simple logic that looks like the following:

```
        if (intent == "atis_flight" or intent == "atis_airline") and len(tokens["LOC"]):
            print ("Can I confirmed if you just asked about flying to " + tokens["LOC"][0])
        elif intent == "atis_yes":
            print ("Great, then let's me book the ticket for you")
        elif intent == "atis_no":
            print ("Oh I am sorry what did I get wrong?")
        elif intent == "atis_greeting":
            print ("Hi, how are you?")            
        else:
            print ("I don't quite know how to respond to " + intent + " yet.")
```

In [ ]:
def chatbot():
    print ("Chatbot Started. Press 'Q'+Enter to quit.")

    while (True):
        input_text = input()
        if input_text == "Q" or input_text == "":
            break

        intent = infer_intent(input_text)
        tokens = infer_combined_tokens(input_text)

        # TODO: 
        # Write you own logic to conduct a conversation with the user
        # about buying tickets and flying somewhere.
        #...#

        
        


    print ("Good bye!")

chatbot()
